<a href="https://colab.research.google.com/github/flying-spagetti/TextRoid/blob/main/NextWordPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [ ]:
model = TFGPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer.save_pretrained("/gpt2_tokenizer")
model.save_pretrained("/gpt2_model")


In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model='gpt2')
generator("Hello, I'm a language model,", max_length=10, num_return_sequences=3)


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, let do"},
 {'generated_text': "Hello, I'm a language model, a language"},
 {'generated_text': "Hello, I'm a language model, so I"}]

In [ ]:
def get_predictions_from_model(input_text):
    input_text = input_text.strip()
    inputs = tokenizer.encode(input_text, return_tensors="tf")
    with tf.device("/CPU:0"):  # Ensure CPU usage for inference
        predictions = model(inputs).logits
    return predictions


def get_next_word_probabilities(sentence, top_k):
    # Get the model predictions for the sentence.
    predictions = get_predictions_from_model(sentence)

    # Get the next token candidates.
    next_token_candidates_tensor = predictions[0, -1, :]

    # Get the top k next token candidates.
    topk_candidates_indexes = tf.argsort(next_token_candidates_tensor, direction='DESCENDING')[:top_k]
    topk_candidates_indexes = tf.sort(topk_candidates_indexes)  # Sort for consistent results

    # Get the token probabilities for all candidates.
    all_candidates_probabilities = tf.nn.softmax(next_token_candidates_tensor, axis=-1)

    # Filter the token probabilities for the top k candidates.
    topk_candidates_probabilities = tf.gather(all_candidates_probabilities, topk_candidates_indexes).numpy()

    # Decode the top k candidates back to words.
    topk_candidates_tokens = [tokenizer.decode([idx]).strip() for idx in topk_candidates_indexes]

    # Return the top k candidates and their probabilities as a list of tuples.
    return list(zip(topk_candidates_tokens, topk_candidates_probabilities))

In [ ]:
def predict(sentence):
    sentence = sentence.strip()
    # Get the top 3 candidates and their probabilities
    top_k_candidates = get_next_word_probabilities(sentence, 3)

    # Organize the results
    output = {probability.round(2): {'word': candidate, 'probability': str(probability.round(2))}
              for candidate, probability in top_k_candidates}

    # Sort and return the results
    return [value for _, value in sorted(output.items(), reverse=True)]

In [ ]:
text = 'I love this. I am really'
response_list = predict(text)
print(text)
print(response_list)

I love this. I am really
[{'word': 'happy', 'probability': '0.13'}, {'word': 'excited', 'probability': '0.12'}, {'word': 'looking', 'probability': '0.07'}]


In [ ]:
from transformers import pipeline

In [ ]:
# Install required libraries
!pip install transformers datasets sentence-transformers rouge-score nltk psutil language-tool-python

# Import necessary libraries
import time
import psutil
import os
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
import language_tool_python

# Load the model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = pipeline("text-generation", model=model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:


# Example dataset for evaluation
dataset = [
    {"input": "The capital of France is", "expected": "Paris"},
    {"input": "What is AI?", "expected": "Artificial Intelligence"},
    {"input": "Once upon a time", "expected": "there was a brave knight"},
]

# Metric 1: Factual Accuracy
def test_factual_accuracy(model, dataset):
    correct = 0
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        if item["expected"].lower() in output.lower():
            correct += 1
    return correct / len(dataset)

# Metric 2: Semantic Similarity
embedder = SentenceTransformer('all-MiniLM-L6-v2')

def test_semantic_similarity(model, dataset):
    similarities = []
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        embeddings = embedder.encode([output, item["expected"]])
        similarity = util.cos_sim(embeddings[0], embeddings[1]).item()
        similarities.append(similarity)
    return sum(similarities) / len(similarities)

# Metric 3: Perplexity
def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model.model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    return torch.exp(loss).item()

# Metric 4: Grammar and Syntax
tool = language_tool_python.LanguageTool('en-US')

def test_grammar(text):
    matches = tool.check(text)
    return len(matches)

# Metric 5: Inference Time
def test_inference_time(model, input_text):
    start_time = time.time()
    _ = model(input_text, max_length=50)
    end_time = time.time()
    return end_time - start_time

# Metric 6: Memory Usage
def test_resource_usage(model, input_text):
    process = psutil.Process(os.getpid())
    start_memory = process.memory_info().rss / 1e6  # Convert to MB
    _ = model(input_text, max_length=50)
    end_memory = process.memory_info().rss / 1e6
    return end_memory - start_memory

# Metric 7: Hallucination Rate
def test_hallucination_rate(model, dataset):
    hallucinations = 0
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        if item["expected"].lower() not in output.lower():
            hallucinations += 1
    return hallucinations / len(dataset)

# Metric 8: Summarization Quality (ROUGE Score)
scorer = rouge_scorer.RougeScorer(['rouge-1', 'rouge-2', 'rouge-l'], use_stemmer=True)

def test_summarization_quality(model, dataset):
    scores = []
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        score = scorer.score(item["expected"], output)
        scores.append(score['rouge-l'].fmeasure)
    return sum(scores) / len(scores)

# Combine all metrics into a single evaluation function
def evaluate_model(model, dataset):
    results = {
        "Factual Accuracy (%)": test_factual_accuracy(model, dataset) * 100,
        "Semantic Similarity": test_semantic_similarity(model, dataset),
        "Perplexity": calculate_perplexity(dataset[0]["input"]),
        "Grammar Errors": test_grammar(model(dataset[0]["input"], max_length=50)[0]['generated_text']),
        "Inference Time (seconds)": test_inference_time(model, dataset[0]["input"]),
        "Memory Usage (MB)": test_resource_usage(model, dataset[0]["input"]),
        "Hallucination Rate (%)": test_hallucination_rate(model, dataset) * 100,
        "Summarization Quality (ROUGE-L)": test_summarization_quality(model, dataset),
    }
    return results

# Run the evaluation
results = evaluate_model(model, dataset)

# Print the results
print("Evaluation Results:")
for metric, value in results.items():
    print(f"{metric}: {value:.2f}")

NameError: name 'SentenceTransformer' is not defined

In [1]:

# Load the model


model_name = "distilgpt2"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = pipeline("text-generation", model=model_name)


# Example dataset for evaluation
dataset = [
    {"input": "The capital of France is", "expected": "Paris"},
    {"input": "What is AI?", "expected": "Artificial Intelligence"},
    {"input": "Once upon a time", "expected": "there was a brave knight"},
]

# Metric 1: Factual Accuracy
def test_factual_accuracy(model, dataset):
    correct = 0
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        if item["expected"].lower() in output.lower():
            correct += 1
    return correct / len(dataset)

# Metric 2: Semantic Similarity
embedder = SentenceTransformer('all-MiniLM-L6-v2')

def test_semantic_similarity(model, dataset):
    similarities = []
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        embeddings = embedder.encode([output, item["expected"]])
        similarity = util.cos_sim(embeddings[0], embeddings[1]).item()
        similarities.append(similarity)
    return sum(similarities) / len(similarities)

# Metric 3: Perplexity
def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model.model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    return torch.exp(loss).item()

# Metric 4: Grammar and Syntax
tool = language_tool_python.LanguageTool('en-US')

def test_grammar(text):
    matches = tool.check(text)
    return len(matches)

# Metric 5: Inference Time
def test_inference_time(model, input_text):
    start_time = time.time()
    _ = model(input_text, max_length=50)
    end_time = time.time()
    return end_time - start_time

# Metric 6: Memory Usage
def test_resource_usage(model, input_text):
    process = psutil.Process(os.getpid())
    start_memory = process.memory_info().rss / 1e6  # Convert to MB
    _ = model(input_text, max_length=50)
    end_memory = process.memory_info().rss / 1e6
    return end_memory - start_memory

# Metric 7: Hallucination Rate
def test_hallucination_rate(model, dataset):
    hallucinations = 0
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        if item["expected"].lower() not in output.lower():
            hallucinations += 1
    return hallucinations / len(dataset)

# Metric 8: Summarization Quality (ROUGE Score)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def test_summarization_quality(model, dataset):
    scores = []
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        score = scorer.score(item["expected"], output)
        scores.append(score['rougeL'].fmeasure)
    return sum(scores) / len(scores)

# Combine all metrics into a single evaluation function
def evaluate_model(model, dataset):
    results = {
        "Factual Accuracy (%)": test_factual_accuracy(model, dataset) * 100,
        "Semantic Similarity": test_semantic_similarity(model, dataset),
        "Perplexity": calculate_perplexity(dataset[0]["input"]),
        "Grammar Errors": test_grammar(model(dataset[0]["input"], max_length=50)[0]['generated_text']),
        "Inference Time (seconds)": test_inference_time(model, dataset[0]["input"]),
        "Memory Usage (MB)": test_resource_usage(model, dataset[0]["input"]),
        "Hallucination Rate (%)": test_hallucination_rate(model, dataset) * 100,
        "Summarization Quality (ROUGE-L)": test_summarization_quality(model, dataset),
    }
    return results

# Run the evaluation
results = evaluate_model(model, dataset)

# Print the results
print("Evaluation Results:")
for metric, value in results.items():
    print(f"{metric}: {value:.2f}")

NameError: name 'AutoTokenizer' is not defined

In [ ]:
from huggingface_hub import login
login()  # Enter your Hugging Face token when prompted

In [ ]:
# Install required libraries
!pip install transformers datasets sentence-transformers rouge-score nltk psutil language-tool-python

# Import necessary libraries
import time
import psutil
import os
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
import language_tool_python

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
 

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Install required libraries
!pip install transformers datasets sentence-transformers rouge-score nltk psutil language-tool-python

# Import necessary libraries
import time
import psutil
import os
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
import language_tool_python

# Load the model
model_name = "distilgpt2"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
# Example dataset for evaluation
dataset = [
    # Factual Queries
    {"input": "The capital of France is", "expected": "Paris"},
    {"input": "The largest planet in the solar system is", "expected": "Jupiter"},
    {"input": "The chemical symbol for water is", "expected": "H2O"},
    {"input": "The first President of the United States was", "expected": "George Washington"},
    {"input": "Photosynthesis occurs in which part of the plant?", "expected": "leaves"},

    # Creative Writing Prompts
    {"input": "Once upon a time", "expected": "in a faraway land"},
    {"input": "In a world where humans could fly", "expected": "cities were built in the skies"},
    {"input": "Deep in the heart of the forest", "expected": "a mysterious light flickered"},
    {"input": "The old bookshelf creaked as it revealed", "expected": "a hidden door behind it"},
    {"input": "As the sun set over the horizon", "expected": "the sky turned shades of orange and pink"},

    # Conversational Inputs
    {"input": "Hello, how are you?", "expected": "I'm doing well, thank you."},
    {"input": "What's your favorite color?", "expected": "I don't have preferences, but I think blue is nice."},
    {"input": "Can you help me with this?", "expected": "Of course! What do you need help with?"},
    {"input": "Tell me something interesting.", "expected": "Did you know octopuses have three hearts?"},
    {"input": "What's the weather like today?", "expected": "I'm not sure, but you can check a weather app."},

    # Edge Cases
    {"input": "AI refers to", "expected": "Artificial Intelligence"},
    {"input": "The square root of 64 is", "expected": "8"},
    {"input": "A triangle has", "expected": "three sides"},
    {"input": "The opposite of hot is", "expected": "cold"},
    {"input": "If it rains, I will", "expected": "stay indoors"},

    # Multilingual Support (Optional)
    {"input": "La capital de Francia es", "expected": "París"},
    {"input": "Quelle est la capitale de la France ?", "expected": "Paris"},
    {"input": "Was ist KI?", "expected": "Künstliche Intelligenz"},
]

# Metric 1: Factual Accuracy
def test_factual_accuracy(model, dataset):
    correct = 0
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        if item["expected"].lower() in output.lower():
            correct += 1
    return correct / len(dataset)

# Metric 2: Semantic Similarity
embedder = SentenceTransformer('all-MiniLM-L6-v2')

def test_semantic_similarity(model, dataset):
    similarities = []
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        embeddings = embedder.encode([output, item["expected"]])
        similarity = util.cos_sim(embeddings[0], embeddings[1]).item()
        similarities.append(similarity)
    return sum(similarities) / len(similarities)

# Metric 3: Perplexity
def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)  # Move inputs to GPU
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    return torch.exp(loss).item()

# Metric 4: Grammar and Syntax
tool = language_tool_python.LanguageTool('en-US')

def test_grammar(text):
    matches = tool.check(text)
    return len(matches)

# Metric 5: Inference Time
def test_inference_time(model, input_text):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)  # Move inputs to GPU
    start_time = time.time()
    _ = model.generate(**inputs, max_length=50)
    end_time = time.time()
    return end_time - start_time

# Metric 6: Memory Usage
def test_resource_usage(model, input_text):
    process = psutil.Process(os.getpid())
    start_memory = process.memory_info().rss / 1e6  # Convert to MB
    _ = model(input_text, max_length=50)
    end_memory = process.memory_info().rss / 1e6
    return end_memory - start_memory

# Metric 7: Hallucination Rate
def test_hallucination_rate(model, dataset):
    hallucinations = 0
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        if item["expected"].lower() not in output.lower():
            hallucinations += 1
    return hallucinations / len(dataset)

# Metric 8: Summarization Quality (ROUGE Score)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def test_summarization_quality(model, dataset):
    scores = []
    for item in dataset:
        output = model(item["input"], max_length=50)[0]['generated_text']
        score = scorer.score(item["expected"], output)
        scores.append(score['rougeL'].fmeasure)
    return sum(scores) / len(scores)

# Combine all metrics into a single evaluation function
def evaluate_model(model, dataset):
    results = {
        "Factual Accuracy (%)": test_factual_accuracy(model, dataset) * 100,
        "Semantic Similarity": test_semantic_similarity(model, dataset),
        "Perplexity": calculate_perplexity(dataset[0]["input"]),
        "Grammar Errors": test_grammar(model(dataset[0]["input"], max_length=50)[0]['generated_text']),
        "Inference Time (seconds)": test_inference_time(model, dataset[0]["input"]),
        "Memory Usage (MB)": test_resource_usage(model, dataset[0]["input"]),
        "Hallucination Rate (%)": test_hallucination_rate(model, dataset) * 100,
        "Summarization Quality (ROUGE-L)": test_summarization_quality(model, dataset),
    }
    return results

# Run the evaluation
results = evaluate_model(model, dataset)

# Print the results
print("Evaluation Results:")
for metric, value in results.items():
    print(f"{metric}: {value:.2f}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpz0382uh0.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.5.zip to /root/.cache/language_tool_python.


AttributeError: 'str' object has no attribute 'size'

In [2]:
# Install required libraries
!pip install transformers datasets sentence-transformers rouge-score nltk psutil language-tool-python

# Import necessary libraries
import time
import psutil
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
import language_tool_python

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the model and tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Example dataset for evaluation
dataset = [
    # Factual Queries
    {"input": "The capital of France is", "expected": "Paris"},
    {"input": "The largest planet in the solar system is", "expected": "Jupiter"},
    {"input": "The chemical symbol for water is", "expected": "H2O"},
    {"input": "The first President of the United States was", "expected": "George Washington"},
    {"input": "Photosynthesis occurs in which part of the plant?", "expected": "leaves"},
    # Creative Writing Prompts
    {"input": "Once upon a time", "expected": "in a faraway land"},
    {"input": "In a world where humans could fly", "expected": "cities were built in the skies"},
    {"input": "Deep in the heart of the forest", "expected": "a mysterious light flickered"},
    {"input": "The old bookshelf creaked as it revealed", "expected": "a hidden door behind it"},
    {"input": "As the sun set over the horizon", "expected": "the sky turned shades of orange and pink"},
    # Conversational Inputs
    {"input": "Hello, how are you?", "expected": "I'm doing well, thank you."},
    {"input": "What's your favorite color?", "expected": "I don't have preferences, but I think blue is nice."},
    {"input": "Can you help me with this?", "expected": "Of course! What do you need help with?"},
    {"input": "Tell me something interesting.", "expected": "Did you know octopuses have three hearts?"},
    {"input": "What's the weather like today?", "expected": "I'm not sure, but you can check a weather app."},
    # Edge Cases
    {"input": "AI refers to", "expected": "Artificial Intelligence"},
    {"input": "The square root of 64 is", "expected": "8"},
    {"input": "A triangle has", "expected": "three sides"},
    {"input": "The opposite of hot is", "expected": "cold"},
    {"input": "If it rains, I will", "expected": "stay indoors"},
    # Multilingual Support (Optional)
    {"input": "La capital de Francia es", "expected": "París"},
    {"input": "Quelle est la capitale de la France ?", "expected": "Paris"},
    {"input": "Was ist KI?", "expected": "Künstliche Intelligenz"},
]

# Metric 1: Factual Accuracy
def test_factual_accuracy(model, tokenizer, dataset):
    correct = 0
    for item in dataset:
        inputs = tokenizer(item["input"], return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_length=50)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if item["expected"].lower() in generated_text.lower():
            correct += 1
    return correct / len(dataset)

# Metric 2: Semantic Similarity
embedder = SentenceTransformer('all-MiniLM-L6-v2')
def test_semantic_similarity(model, tokenizer, dataset):
    similarities = []
    for item in dataset:
        inputs = tokenizer(item["input"], return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_length=50)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        embeddings = embedder.encode([generated_text, item["expected"]])
        similarity = util.cos_sim(embeddings[0], embeddings[1]).item()
        similarities.append(similarity)
    return sum(similarities) / len(similarities)

# Metric 3: Perplexity
def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    labels = inputs["input_ids"]
    with torch.no_grad():
        outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    return torch.exp(loss).item()

# Metric 4: Grammar and Syntax
tool = language_tool_python.LanguageTool('en-US')
def test_grammar(text):
    matches = tool.check(text)
    return len(matches)

# Metric 5: Inference Time
def test_inference_time(model, tokenizer, input_text):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    start_time = time.time()
    _ = model.generate(**inputs, max_length=50)
    end_time = time.time()
    return end_time - start_time

# Metric 6: Memory Usage
def test_resource_usage(model, tokenizer, input_text):
    process = psutil.Process(os.getpid())
    start_memory = process.memory_info().rss / 1e6  # Convert to MB
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    _ = model.generate(**inputs, max_length=50)
    end_memory = process.memory_info().rss / 1e6
    return end_memory - start_memory

# Metric 7: Hallucination Rate
def test_hallucination_rate(model, tokenizer, dataset):
    hallucinations = 0
    for item in dataset:
        inputs = tokenizer(item["input"], return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_length=50)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if item["expected"].lower() not in generated_text.lower():
            hallucinations += 1
    return hallucinations / len(dataset)

# Metric 8: Summarization Quality (ROUGE Score)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
def test_summarization_quality(model, tokenizer, dataset):
    scores = []
    for item in dataset:
        inputs = tokenizer(item["input"], return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_length=50)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        score = scorer.score(item["expected"], generated_text)
        scores.append(score['rougeL'].fmeasure)
    return sum(scores) / len(scores)

# Combine all metrics into a single evaluation function
def evaluate_model(model, tokenizer, dataset):
    results = {
        "Factual Accuracy (%)": test_factual_accuracy(model, tokenizer, dataset) * 100,
        "Semantic Similarity": test_semantic_similarity(model, tokenizer, dataset),
        "Perplexity": calculate_perplexity(model, tokenizer, dataset[0]["input"]),
        "Grammar Errors": test_grammar(tokenizer.decode(model.generate(
            **tokenizer(dataset[0]["input"], return_tensors="pt").to(device), max_length=50)[0],
            skip_special_tokens=True)),
        "Inference Time (seconds)": test_inference_time(model, tokenizer, dataset[0]["input"]),
        "Memory Usage (MB)": test_resource_usage(model, tokenizer, dataset[0]["input"]),
        "Hallucination Rate (%)": test_hallucination_rate(model, tokenizer, dataset) * 100,
        "Summarization Quality (ROUGE-L)": test_summarization_quality(model, tokenizer, dataset),
    }
    return results

# Run the evaluation
results = evaluate_model(model, tokenizer, dataset)

# Print the results
print("Evaluation Results:")
for metric, value in results.items():
    print(f"{metric}: {value:.2f}")

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Evaluation Results:
Factual Accuracy (%): 39.13
Semantic Similarity: 0.37
Perplexity: 143.05
Grammar Errors: 0.00
Inference Time (seconds): 1.52
Memory Usage (MB): 0.00
Hallucination Rate (%): 60.87
Summarization Quality (ROUGE-L): 0.09
